In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [207]:
zakupki = pd.read_csv('100Zakupok.csv', header=1,dtype={'телефон':'object'})
zakupki['телефон'].fillna(0, inplace=True)
zakupki['телефон'] = pd.to_numeric(zakupki['телефон'])
zakupki.rename(columns={'телефон':'phone'}, inplace=True)
zakupki.drop(zakupki[(zakupki['Закупка']=='Пристрой')|(zakupki['Закупка']=='Переплата')].index, inplace=True)

In [208]:
zakupki_bot = pd.read_excel('100 закупок (боты).xlsx', sheet_name='tg')

In [209]:
df_zakup = pd.merge(zakupki[['Дата заказа','phone']],zakupki_bot[['phone','regdate','name']],on='phone')

In [210]:
df_zakup['Дата заказа'] = pd.to_datetime(df_zakup['Дата заказа'])
df_zakup['regdate'] = pd.to_datetime(df_zakup['regdate']).dt.normalize()

In [211]:
df_zakup = df_zakup.groupby(['phone','regdate']).agg({'Дата заказа':'min'}).reset_index()

In [212]:
df_zakup['days_to_first'] = (df_zakup['Дата заказа'] - df_zakup['regdate'])
df_zakup['days_to_first'] = (df_zakup['days_to_first']/np.timedelta64(1, 'D')).astype(int)
df_zakup = df_zakup.sort_values('days_to_first', ascending=False)
#df_zakup.to_excel('Результат.xlsx',sheet_name='количествод дней')

In [213]:
from_viber = df_zakup.value_counts(df_zakup['days_to_first']<0)
from_viber

days_to_first
True     176
False    103
dtype: int64

In [237]:
df_zakup_1 = df_zakup[(df_zakup['days_to_first']>=0)]

In [218]:
df_referal = pd.merge(zakupki_bot[['phone','regdate','referal']],df_zakup_1['phone'], on='phone')
df_referal.fillna('Нет реферала', inplace=True)
df_referal_count = df_referal.groupby('referal')['phone'].count()
#df_referal_count.to_excel('Результат.xlsx',sheet_name='реферал')

In [235]:
df_second = pd.merge(zakupki[['Закупка','Дата заказа','phone']],df_zakup_1['phone'], on='phone')
df_second['Дата заказа'] = pd.to_datetime(df_second['Дата заказа'])
df_second_1 = df_second.groupby(['phone','Закупка']).agg({'Дата заказа':'min'}).reset_index()
df_second = df_second_1.groupby('phone')['Закупка'].count().sort_values(ascending=False)

In [236]:
writer = pd.ExcelWriter('Результат.xlsx', engine='xlsxwriter')

#store your dataframes in a  dict, where the key is the sheet name you want
frames = {'количествод дней': df_zakup, 'реферал': df_referal_count, 'количество закупок':df_second,
          'закупки по датам':df_second_1}

#now loop thru and put each on a specific sheet
for sheet, frame in  frames.items(): # .use .items for python 3.X
    frame.to_excel(writer, sheet_name = sheet)

#critical last step
writer.save()